# Automated ML

Import Dependencies. 

In [4]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

#!pip install azureml-sdk==1.19.0

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
I will be using a diabetes dataset from datahub.io dataset on gestational diabetes. https://datahub.io/machine-learning/diabetes/r/diabetes.csv
This dataset will aid to forecast gestational diabetes in women.


In [5]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "pimadiabetes"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V12',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

###################
# Define Dataset
###################
found = False
key = "pimadiabetes"
description_text = "Capstone Project - Forecast Diabetes"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://datahub.io/machine-learning/diabetes/r/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

trn-aue-aa-ml
trn-aue-advanced-analytics-ml
australiaeast
8a086c9c-5530-4791-89af-62c8cdfda3fd
Found existing cluster, use it.
Succeeded....................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


,preg,plas,pres,skin,insu,mass,pedi,age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [6]:
# choose a name for experiment
experiment_name = 'automlpimadiabetesNoFeat'
project_folder = './Augusto_Avila/capstone'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

Choosing a classification model to predict the likelihood of gestational diabetes, the variable class will hold the forecasted value and will be using the 
AUC_weighed metric as the result of initial automl experiment execution as the primary metric

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=amlcompute_cluster_name,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="class",   
                             path = project_folder,
                             enable_early_stopping= True,
                             enable_onnx_compatible_models=True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [8]:
## Submit Experiment

from azureml.widgets import RunDetails
from azureml.core.run import Run

remote_run = experiment.submit(automl_config,show_output=True)


Running on remote.
No run_configuration provided, running on pimadiabetes with default configuration
Running on remote compute: pimadiabetes
Parent Run ID: AutoML_fb57bb5b-6d97-45b1-aa8c-de8c5dc27fca

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TY

## Run Details



In [10]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
print(remote_run.get_portal_url())
remote_run.wait_for_completion(show_output=True)


best_run, fitted_model = remote_run.get_output()

fitted_model.steps[1][1].estimators

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

https://ml.azure.com/experiments/automlpimadiabetesNoFeat/runs/AutoML_fb57bb5b-6d97-45b1-aa8c-de8c5dc27fca?wsid=/subscriptions/8a086c9c-5530-4791-89af-62c8cdfda3fd/resourcegroups/trn-aue-advanced-analytics-ml/workspaces/trn-aue-aa-ml


****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbala

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0.post1
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.18.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


[('1',
  Pipeline(memory=None,
           steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                  ('xgboostclassifier',
                   XGBoostClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1, missing=nan,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, tree_method='auto', verbose=-10,
                                     verbosity=0))],
           verbose=False)),
 ('3',
  Pipeline(memory=N

In [16]:
# Print Best Run

print(best_run)

print(best_run.properties['score'])

Run(Experiment: automlpimadiabetesNoFeat,
Id: AutoML_fb57bb5b-6d97-45b1-aa8c-de8c5dc27fca_38,
Type: azureml.scriptrun,
Status: Completed)
0.8468314726552164


In [27]:
# Print Fitted Model

print(fitted_model)

Pipeline(memory=None,
         steps=[('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=None,
                                               estimators=[('34',
                                                            Pipeline(memory=None,
                                                                     steps=[('MinMaxScaler',
                                                                             MinMaxScaler(copy=True,
                                                                                          feature_range=(0,
                                                                                                         1))),
                                                                            ('ExtraTreesClassifier',
                                                                             ExtraTreesClassifier(bootstrap=True,
                                                                             

## Best Model

Get the best model from the automl experiments and display all the properties of the model.



In [17]:
import joblib
import azureml.train.automl

# Get your best run and save the model from that run.

#!pip install azureml-train-automl-runtime==1.20.0

#!pip install azureml-sklearn

if "output" not in os.listdir():
   os.mkdir("./output")   

remote_run.get_metrics('AUC_weighted')

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print ('Best run id:',best_run.id)
print ('################################')
print ('\n AUC_weighted:',best_run_metrics['AUC_weighted'])
print ('################################')
print ('\n Learning rate:', parameter_values)
print ('################################')


best_run.get_details()
best_run.get_tags()


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0.post1
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.18.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automlpimadiabetesNoFeat,
Id: AutoML_fb57bb5b-6d97-45b1-aa8c-de8c5dc27fca_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    oob_score=False,
                                                                                                    random_state=None,
                  

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":5,"CurrentNodeCount":5}',
 'ensembled_iterations': '[1, 3, 21, 2, 32, 15, 26, 33, 4, 18, 10]',
 'ensembled_algorithms': "['XGBoostClassifier', 'RandomForest', 'XGBoostClassifier', 'RandomForest', 'LogisticRegression', 'ExtremeRandomTrees', 'GradientBoosting', 'LogisticRegression', 'RandomForest', 'RandomForest', 'RandomForest']",
 'ensemble_weights': '[0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]',
 'best_individual_pipeline_score': '0.8366210317228242',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_fb57bb5b-6d97-45b1-aa8c-de8c5dc27fca_ModelExplain',
 'model_

In [8]:

best_run.get_file_names()
joblib.dump(parameter_values,filename='./output/Capstone_automl_best.joblib')

['./output/Capstone_automl_best.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.


In [48]:

model_dep = remote_run.register_model(model_name='automlpimadiabetes',
                           tags={'Name': 'Pima Diabetes Best AutoML Model'})#,
                           #model_path='outputs/automldiabetesbmodel.pkl')
print(model_dep.name, model_dep.id, model_dep.version, sep='\t')

automlpimadiabetes	automlpimadiabetes:3	3


## Register the WebService

In [61]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

bestmodel, fmodel = remote_run.get_output()
bestmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')  
#bestmodel.download_file('outputs/conda_env_v_1_0_0.yml', myenv)  

#env = Environment.get(ws, "AzureML-Minimal").clone('temp')
#env = Environment.get(ws, "AzureML-Minimal",'myenv')
#env = Environment.from_conda_specification(name='automl',file_path='conda_env_v_1_0_0.yml')

env = bestmodel.get_environment()

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='scoring.py',
                                    environment=env)


deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "pimadiabetesendpv3", [model_dep], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................................................................................................

## Test and delete WebService

In [7]:
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.webservice import AciWebservice, Webservice


service = Webservice(workspace=ws, name='pimadiabetesendpv3')
print(service.scoring_uri)
scoring_uri = service.scoring_uri
#print(service.swagger_uri)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# URL for the web service
#scoring_uri = '<your web service URI>'
# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
                1,
                150,
                80,
                29,
                0,
                28.1,
                0.627,
                25,
            ],
            [
               3,
                150,
                80,
                29,
                0,
                28.1,
                0.627,
                44,],
            [
               4,
                150,
                80,
                29,
                0,
                28.1,
                0.627,
                45,]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print (resp)
print(resp.text)

http://159a4ac6-4925-4238-bd97-dbe0430d3a97.australiaeast.azurecontainer.io/score
<Response [200]>
"{\"result\": [\"tested_negative\", \"tested_positive\", \"tested_positive\"]}"


## Enable Logging and Log Capture

In [8]:
service = Webservice(workspace=ws, name='pimadiabetesendpv3')
service.update(enable_app_insights=True)

logs = service.get_logs()
print(logs)

## Delete WebSERVICE

In [69]:
service = Webservice(workspace=ws, name='pimadiabetesendpv2')
service.delete()

## ONNX Model

In [85]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)

onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)


model = Model.register(model_path = "./",
                       model_name = "best_model.onnx",
                       tags = {"onnx": "MyFirstonnx"},
                       description = "pimadiabetesonnx",
                       workspace = ws)



Registering model best_model.onnx


In [27]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

bestmodel, fmodel = remote_run.get_output()
#bestmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py') 
bestmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'myenv.yml')  

In [15]:
from pprint import pprint

def print_model(fitted_model, prefix=""):
    for step in fitted_model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '3', '21', '2', '32', '15', '26', '33', '4', '18', '10'],
 'weights': [0.2857142857142857,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'le